# PCA for Essential Variable Identification + Logistic Regression (Anderson Cancer Center)
**Generated:** 2025-11-07 04:08:27Z (UTC)

This notebook demonstrates how to:
1. Load the Breast Cancer dataset from `sklearn.datasets`
2. Standardize features
3. Apply Principal Component Analysis (PCA) to extract essential components
4. Reduce to **2 principal components**
5. Fit a **Logistic Regression** classifier for prediction
6. Evaluate the model and visualize results

> **Why PCA?** PCA helps identify combinations of variables (principal components) that explain the most variance. This is useful for prioritizing factors that could be compelling for **donor funding** and operational planning.


In [ ]:
# !pip install scikit-learn pandas matplotlib numpy  # Uncomment if running in a fresh environment

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


## 1) Load and inspect the dataset

In [ ]:
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="target")

print(f"Shape: {X.shape}")
X.head()

## 2) Standardize features (important for PCA)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Confirm scaling result: mean ~ 0, std ~ 1 (per feature)
means = X_scaled.mean(axis=0)[:5]
stds = X_scaled.std(axis=0)[:5]
print("First 5 scaled feature means ~ 0:", np.round(means, 4))
print("First 5 scaled feature stds  ~ 1:", np.round(stds, 4))

## 3) Fit PCA and examine explained variance

In [ ]:
pca_full = PCA()
pca_full.fit(X_scaled)

explained_var_ratio = pca_full.explained_variance_ratio_
cum_explained = np.cumsum(explained_var_ratio)

pd.DataFrame({
    "PC": np.arange(1, len(explained_var_ratio)+1),
    "Explained_Var_Ratio": explained_var_ratio,
    "Cumulative_Explained_Var": cum_explained
}).head(10)

In [ ]:
# Scree plot (Explained variance by component)
plt.figure()
plt.plot(np.arange(1, len(explained_var_ratio)+1), explained_var_ratio, marker='o')
plt.title("Scree Plot: Explained Variance Ratio by Principal Component")
plt.xlabel("Principal Component")
plt.ylabel("Explained Variance Ratio")
plt.grid(True)
plt.show()

# Cumulative explained variance plot
plt.figure()
plt.plot(np.arange(1, len(cum_explained)+1), cum_explained, marker='o')
plt.title("Cumulative Explained Variance")
plt.xlabel("Number of Components")
plt.ylabel("Cumulative Explained Variance")
plt.grid(True)
plt.show()

## 4) Reduce to 2 principal components

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(X_pca, columns=["PC1", "PC2"])
pca_df['target'] = y.values
pca_df.head()

In [ ]:
# 2D scatter of the two principal components
plt.figure()
for label, label_name in [(0, 'malignant'), (1, 'benign')]:
    subset = pca_df[pca_df['target'] == label]
    plt.scatter(subset['PC1'], subset['PC2'], label=label_name, alpha=0.7)
plt.legend()
plt.title("PCA (2 Components) - Breast Cancer")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.grid(True)
plt.show()

print("Explained variance by PC1 and PC2:", np.round(pca.explained_variance_ratio_, 4))
print("Total explained variance (2 PCs):", round(pca.explained_variance_ratio_.sum(), 4))

## 5) Train-test split on PCA features

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    pca_df[['PC1', 'PC2']], pca_df['target'], test_size=0.2, random_state=42, stratify=pca_df['target']
)

X_train.shape, X_test.shape

## 6) Fit Logistic Regression and evaluate

In [ ]:
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)
y_proba = log_reg.predict_proba(X_test)[:, 1]

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}\n")
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=data.target_names))

In [ ]:
cm = confusion_matrix(y_test, y_pred)

fig = plt.figure()
plt.imshow(cm, interpolation='nearest')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.xticks([0,1], data.target_names)
plt.yticks([0,1], data.target_names)
for (i, j), val in np.ndenumerate(cm):
    plt.text(j, i, int(val), ha='center', va='center')
plt.colorbar()
plt.tight_layout()
plt.show()

## 7) Interpretation and Next Steps
- **PCA** condensed many correlated features into just **two** informative components.
- Even with only 2 PCs, Logistic Regression can achieve strong performance for this dataset.
- For **donor funding** discussions, you can translate **PC loadings** back to the original variables to identify which clinical measures contribute most to PC1 and PC2.

### Component Loadings (which features drive each PC)
The loading matrix shows how much each original feature contributes to a principal component.


In [ ]:
# Component loadings for the 2-PC model
loadings = pd.DataFrame(
    pca.components_.T,
    columns=['PC1', 'PC2'],
    index=X.columns
).sort_values('PC1', ascending=False)

loadings.head(10)